In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import re

%load_ext autoreload
%autoreload 2
import functions
import warnings
warnings.filterwarnings("ignore")



In [3]:
df_horas_juego = pd.read_parquet("data\horas_juego.parquet")

In [4]:
df_reviews_year = pd.read_parquet("data\df_reviews_year.parquet")

In [5]:
df_horas_juego

,genres,item_id,release_year,publisher,title,item_name,playtime_forever,steam_id,items_count,user_id,user_url
0,Action,282010,1997,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,5,76561198053985682,188,UTNerd24,http://steamcommunity.com/id/UTNerd24
1,Action,282010,1997,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,0,76561198067520555,154,I_DID_911_JUST_SAYING,http://steamcommunity.com/id/I_DID_911_JUST_SA...
2,Action,282010,1997,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,0,76561197962104795,79,76561197962104795,http://steamcommunity.com/profiles/76561197962...
3,Action,282010,1997,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,0,76561198011684252,331,r3ap3r78,http://steamcommunity.com/id/r3ap3r78
4,Action,282010,1997,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,13,76561198072532181,583,saint556,http://steamcommunity.com/id/saint556
...,...,...,...,...,...,...,...,...,...,...,...
9877299,Action,80,2004,Valve,Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,0,76561198273508956,15,76561198273508956,http://steamcommunity.com/profiles/76561198273...
9877300,Action,80,2004,Valve,Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,0,76561198282090798,5,76561198282090798,http://steamcommunity.com/profiles/76561198282...
9877301,Action,80,2004,Valve,Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,0,76561198283103833,58,943525,http://steamcommunity.com/id/943525
9877302,Action,80,2004,Valve,Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,9,76561198283312749,49,76561198283312749,http://steamcommunity.com/profiles/76561198283...


In [6]:
def PlayTimeGenre(genero):
    gener = df_horas_juego[df_horas_juego["genres"].str.lower() ==genero.lower()]
    if not gener.empty:
        horas_anio = gener.groupby("release_year")["playtime_forever"].sum().reset_index()
        max_horas  = horas_anio["playtime_forever"].max()
        anio = horas_anio.loc[horas_anio["playtime_forever"] == max_horas,"release_year"].iloc[0]
        return  {anio,"Año de lanzamiento con mas horas jugadas para el Genero:" }

In [7]:
PlayTimeGenre("Racing")

{'2015', 'Año de lanzamiento con mas horas jugadas para el Genero:'}

In [8]:
def UserForGenre(genero):
    # Filtrar las horas de juego para el género especificado
    horas = df_horas_juego[df_horas_juego['genres'] == genero][["playtime_forever", "release_year", "user_id"]]

    # Agrupar por usuario y sumar las horas jugadas
    player = horas.groupby("user_id")["playtime_forever"].sum()

    # Obtener el usuario con más horas jugadas
    player_max = player.idxmax()

    # Obtener la acumulación de horas jugadas por año
    year = horas.groupby("release_year")["playtime_forever"].sum().reset_index()

    # Filtrar las horas jugadas del usuario con más horas
    horas_player = horas[horas["user_id"] == player_max]

    # Crear el diccionario de retorno
    resultado = {
        "Usuario con más horas jugadas para " + genero: player_max,
        "Horas jugadas": year.to_dict(orient="records")
    }

    return resultado

In [9]:
UserForGenre("Action")

{'Usuario con más horas jugadas para Action': 'Sp3ctre',
 'Horas jugadas': [{'release_year': '1983', 'playtime_forever': 3473},
  {'release_year': '1984', 'playtime_forever': 384},
  {'release_year': '1988', 'playtime_forever': 16001},
  {'release_year': '1989', 'playtime_forever': 607},
  {'release_year': '1990', 'playtime_forever': 18335},
  {'release_year': '1991', 'playtime_forever': 2502},
  {'release_year': '1992', 'playtime_forever': 1925},
  {'release_year': '1993', 'playtime_forever': 24623},
  {'release_year': '1994', 'playtime_forever': 120177},
  {'release_year': '1995', 'playtime_forever': 216144},
  {'release_year': '1996', 'playtime_forever': 67844},
  {'release_year': '1997', 'playtime_forever': 359836},
  {'release_year': '1998', 'playtime_forever': 2899116},
  {'release_year': '1999', 'playtime_forever': 2715157},
  {'release_year': '2000', 'playtime_forever': 18357855},
  {'release_year': '2001', 'playtime_forever': 1326310},
  {'release_year': '2002', 'playtime_fore

In [10]:
df_reviews_year

,item_id,title,reviews_posted,reviews_recommend,sentiment_analysis
3,70,Half-Life,2015,True,0
4,70,Half-Life,2014,True,0
8,70,Half-Life,2015,True,2
9,70,Half-Life,2014,True,2
10,70,Half-Life,2011,True,2
...,...,...,...,...,...
122014,80,Counter-Strike: Condition Zero,2014,False,1
122015,80,Counter-Strike: Condition Zero,2013,True,2
122016,80,Counter-Strike: Condition Zero,2011,True,2
122017,80,Counter-Strike: Condition Zero,2014,True,2


In [11]:
def UsersRecommend(anio):

   filtro = (df_reviews_year["reviews_posted"] == anio) & (df_reviews_year["reviews_recommend"] == True) & (df_reviews_year["sentiment_analysis"] >= 1 )
   reviews = df_reviews_year[filtro] 

   games = reviews.groupby(df_reviews_year["item_id"]).size().reset_index(name="count")
   games = games.sort_values(by = "count",ascending = False)
   return games.head(3)
      
   
    

In [12]:
UsersRecommend(2015)

,item_id,count
20,730,1234
971,304930,1140
15,440,1098


In [17]:
def UsersNotRecommend(anio):

    filtro = (df_reviews_year["reviews_posted"] == anio) & (df_reviews_year["reviews_recommend"] == False) & (df_reviews_year["sentiment_analysis"] == 0  )
    reviews = df_reviews_year[filtro] 

    games = reviews.groupby(df_reviews_year["item_id"]).size().reset_index(name="count")
    games = games.sort_values(by = "count",ascending = False)
    resultado = games.head(3)
    top_por_anio = {} 
    for index, row in resultado.iterrows():
       anio_info={
        "item_id": row["item_id"],
        "count": row["count"]
       }
       top_por_anio[index +1 ] = anio_info
    return top_por_anio

In [18]:
UsersNotRecommend(2015)

{133: {'item_id': 221100, 'count': 172},
 203: {'item_id': 252490, 'count': 156},
 360: {'item_id': 346110, 'count': 85}}

In [ ]:
contador = [0,0,0]

In [ ]:
contpos

1

In [5]:
def sentiment_analysis(anio):
    positive = 0
    negative = 0
    neutral = 0
    filtro = (df_reviews_year["reviews_posted"] == anio)
    reviews = df_reviews_year[filtro]
    
    for i in  reviews["sentiment_analysis"]:
        if i == 2:
            positive +=1
        elif i == 1:
            neutral +=1
        elif i == 0:
            negative +=1
    resultado = [
    {"Negativas: ",negative," Positivas: ",positive," Neutrales: ",neutral}]
    return resultado


            




In [6]:
sentiment_analysis(2014)

[{' Neutrales: ', ' Positivas: ', 26915, 9451, 9452, 'Negativas: '}]